# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 2.0.9 which is incompatible.


The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64',no_graphics=True)

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
for brains in env.brain_names:
    print(brains)

ReacherBrain


### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726671e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Loading DDPG Agent


In [8]:
from importlib import reload 
from collections import deque
import ddpg_agent
reload(ddpg_agent)
import torch
from workspace_utils import active_session

In [9]:
def ddpg(n_episodes=2000, max_t=700, agent = None, 
         checkpoint_score = 25, breakpoint_score = 30, filename_prefix = ""):
    
    
    scores_deque = deque(maxlen=100)
    scores = []
    goal_steps = []
    max_score = -np.Inf
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        state = env_info.vector_observations[0]            # get the current state
        agent.reset()
        score = 0
        modified_score = 0
        goal_steps.clear()
        for t in range(max_t):
            action = agent.act(state)
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            if reward > 0.0:
                goal_steps.append(t)
            #modified_reward = 100  if reward > 0.0 else -0.1
            modified_reward = reward
            agent.step(state, action, modified_reward, next_state, done)
            state = next_state
            score += reward
            modified_score += modified_reward

        scores_deque.append(score)
        scores.append(score)
        fname = filename_prefix
        print("goal_steps ", goal_steps)
        print('\rEpisode {}\tAverage Score: {:.2f}\tscore: {:.2f}\tmod_score: {:.2f}:'.
              format(i_episode, np.mean(scores_deque), score, modified_score), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            if np.mean(scores_deque)>=checkpoint_score:
                fname += str(i_episode)
                print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
                torch.save(agent.qnetwork_local.state_dict(), fname + 'checkpoint.pth')
        if np.mean(scores_deque)>=breakpoint_score:
            fname += str(i_episode)
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            torch.save(agent.qnetwork_local.state_dict(),  fname + 'checkpoint.pth')
            break   
    
    return scores



In [10]:
actor_hidden_layer = [64 , 64]
critic_hidden_layer = [128, 128, 64]
agent = ddpg_agent.Agent(state_size=state_size, 
                         action_size=action_size, 
                         random_seed=0,
                         actor_arch=actor_hidden_layer,
                         critic_arch=critic_hidden_layer)

In [11]:
with active_session():
    rr_scores = ddpg( agent = agent) # random replay training

Episode 100	Average Score: 0.10	score: 0.00	mod_score: -70.00	goal steps: [], 42, 43] 67, 68, 69]] 74, 75, 76, 77, 459, 460, 461, 462] 284]7, 635, 636, 637, 638]74, 275], 66, 67, 68, 69, 70, 287, 288, 289, 290, 291, 324, 325, 326, 327, 328]
Episode 146	Average Score: 0.10	score: 0.10	mod_score: 230.30	goal steps: [399, 400, 401]2, 33, 34, 35, 36, 37, 84, 85, 86, 87, 88, 89, 90, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 319, 320, 321]

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

def plot_scores(scores):
    # plot the scores
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

In [ ]:
plot_scores(rr_scores) # random replay scores

When finished, you can close the environment.

In [6]:
env.close()